In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange, HoverTool
from bokeh.models.annotations import Title
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10
import nltk
reset_output()
output_notebook()

Loading BokehJS ...

In [2]:
data = pd.read_csv("./YInt.csv",index_col=0, parse_dates=True, infer_datetime_format=True)
data = data.dropna()
data.head()

,location,account,message
time,,,
2020-04-06 00:00:00,Weston,Opportunities2,"Take advantheeseage of theesehese One, theeser..."
2020-04-06 00:00:00,Southton,LazyBCouch,@WatchesThomasBird fork it you're back in he s...
2020-04-06 00:02:00,Southton,______3333_____,Two month left bedoreefore you loose this vibe...
2020-04-06 00:04:00,Broadview,RasoHorse49,"billeeeeer, i miss ytouou !"
2020-04-06 00:07:00,West Parton,CuriousPlateBobbie_Mallon,You obviously need to use rumble! #rumble #toW...


In [3]:
import re, unicodedata, inflect
import contractions

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in nltk.corpus.stopwords.words('english'):
            new_words.append(word)
    return new_words

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = nltk.WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def porter_stemmer(words):
    """Pasar de plural a singular"""
    ps = nltk.PorterStemmer()
    stem = []
    for word in words:
        stem.append(ps.stem(word))
    return stem
    
def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = lemmatize_verbs(words)
    #words = porter_stemmer(words)
    return words

def normalizar_texto(df, loc=""):
    """
    rw: palabras relevantes
    """
    t_text = ""
    if loc == "":
        data_explorer = df.message
    else:
        data_explorer = df.loc[df.location==loc].message
    for idx in data_explorer:
        t_text = t_text + " " + idx 

    t_text = re.sub('\[[^]]*\]', '', t_text)
    t_text = contractions.fix(t_text)
    words = nltk.word_tokenize(t_text)
    words = normalize(words)
    
    return words

In [4]:
pd.unique(data.location)

array(['Weston', 'Southton', 'Broadview', 'West Parton', 'Old Town',
       'Terrapin Springs', 'Downtown', 'Southwest', 'Scenic Vista',
       'East Parton', 'Cheddarford', 'Palace Hills', 'Safe Town',
       'Easton', 'Chapparal', 'Northwest', 'Oak Willow', 'Pepper Mill',
       'Wilson Forest', 'UNKNOWN', '<Location with-held due to contract>'],
      dtype=object)

In [5]:
words = normalizar_texto(data.loc["2020-04-06 19:40:00":"2020-04-06 20:40:00"], loc="Cheddarford")

In [6]:
d = dict()
for i in words:
    if i in d:
        d[i] += 1
    else:
        d.update({i:1})

In [7]:
c_power = ["power", "energy", "electric", "electrician", "electricity", "powerlines", "electric"]
c_shake = ["shake", "earthquake", "quake", "tremble", "rubble", "earthquakes"]
c_hospital = ["hospital", "hospitals", "die", "dead", "death"]
c_disaster = ["disaster", "disasters"]
c_sewer = ["sewer", "sewage"]
c_evacuation = ["evacuation", "evacuate"]
others = ["nuclear", "repair", "structutral", "rescue", "emergency", "dangerous", "problem", "cry",
          "apocalypse", "sad", "ambulance", "victims", "help", "tragedy", "destroy", "roads", "scar", "hazard",
          "bridge", "damage", "build", "radiation", "crew", "shudder", "incident", "wobble"]

p_usar = c_power + c_shake + c_hospital + c_disaster + c_sewer + c_evacuation + others
len(p_usar)

50

In [8]:
list_palavras = ""
bairro= "Weston"
l = []
dbairro = data.loc[data.location==bairro]
for idx, cor in dbairro.iterrows():
    for word in p_usar:
        if word in cor.message:
            if word in c_power:
                list_palavras = list_palavras + " " + "power"
            elif word in c_shake:
                list_palavras = list_palavras + " " + "earthquake"
            elif word in c_hospital:
                list_palavras = list_palavras + " " + "hospital"
            elif word in c_disaster:
                list_palavras = list_palavras + " " + "disaster"
            elif word in c_hospital:
                list_palavras = list_palavras + " " + "hospital"
            elif word in c_sewer:
                list_palavras = list_palavras + " " + "sewer"
            else:
                list_palavras = list_palavras + " " + word
            l.append(word)

In [9]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

def transform_format(val):
	if val == 0:
		return 255
	else:
		return val

wine_mask = np.array(Image.open("map_white.png"))
 
wordcloud = WordCloud(max_font_size=200, max_words=50, mode='RGB',
	mask=wine_mask,
	colormap=plt.cm.Set1,
                      collocations=False,
	contour_width=8, relative_scaling=.5,
	background_color="white").generate(list_palavras)

# Display the generated image:
plt.figure(figsize=(10,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
plt.savefig("wordcloud");

In [13]:
pd.unique(data_norm.location)

array(['Southton', 'Southwest', 'Cheddarford', 'Terrapin Springs',
       'Weston', 'Palace Hills', 'Downtown', 'Easton', 'Old Town',
       'Northwest', 'Safe Town', 'Pepper Mill', 'Scenic Vista',
       'West Parton', 'Oak Willow', 'East Parton', 'Broadview',
       'Chapparal', 'UNKNOWN', '<Location with-held due to contract>',
       'Wilson Forest'], dtype=object)

In [21]:
data_norm = pd.read_csv("./data/data_geral_normalizada.csv", parse_dates=True, infer_datetime_format=True)
data_norm = data_norm.set_index("time")
data_norm.index = pd.to_datetime(data_norm.index)
data_norm.loc[data_norm.location=="Southwest"].loc["2020-04-06 19:40:00":"2020-04-07 20:40:00"]

,location,account,message
time,,,
2020-04-06 21:07:00,Southwest,RunsMargo_BiermannHorse,"['fifty-five', 'k', 'year', 'everything', 'go'..."
2020-04-06 23:14:00,Southwest,Kelly_StewartWhale1986,"['skeptical', 'know', 'shirt', 'nuclear', 'pow..."
2020-04-07 04:02:00,Southwest,handle,"['new', 'tremble', 'better', 'deal', 'ask']"
2020-04-07 05:36:00,Southwest,Jose32Francis,"['someone', 'day', 'hospital']"
2020-04-07 05:59:00,Southwest,FearsEmiliaWhale,"['fork', 'problem', 'guy', 'four', 'five', 'ho..."
2020-04-07 06:10:00,Southwest,FearsEmiliaWhale,"['definitely', 'go', 'snatch', 'rumble', 'shake']"
2020-04-07 07:28:00,Southwest,TallCCar,"['one', 'bite', 'warseas', 'prove', 'dead', 'w..."
2020-04-07 07:48:00,Southwest,BurtonWhale84,"['idiotic', 'things', 'do', 'fine', 'fork', 'n..."
2020-04-07 07:57:00,Southwest,NAttentativeRobinson,"['klingons', 'someone', 'buy', 'prefer', 'cut'..."


### Gráfico final

In [31]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange, HoverTool
from bokeh.models.annotations import Title
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput, RangeSlider, Select
from bokeh.transform import cumsum
from bokeh.palettes import Category10, Category20
import nltk

#Se cargan los datos
data = pd.read_csv("./data/data_procesada.csv", parse_dates=True, infer_datetime_format=True)
data = data.set_index("time")
data.index = pd.to_datetime(data.index)

data_norm = pd.read_csv("./data/data_geral_normalizada.csv", parse_dates=True, infer_datetime_format=True)
data_norm = data_norm.set_index("time")
data_norm.index = pd.to_datetime(data_norm.index)

def histogram(df, col, bins=30):
    edges = []
    hist = []
    largura = max(df.index) - min(df.index)
    h = largura/bins
    aux = min(df.index)
    for i in range(100):
        edges.append(aux)
        hist.append(df.loc[aux:aux + h].count()[col])
        aux = aux + h
    edges.append(aux)
    
    return edges, hist

#Distribução geral dos dados
p1 = figure(plot_height=300, plot_width=800, title="Quantidade de tweets por intervalo de tempo para todos os bairros",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime')
p1.xaxis.axis_label = "dias registrados"
p1.yaxis.axis_label = "quantidade de tweets"
#quantidade de tweets por bairro
p2 = figure(plot_height=300, plot_width=300, title="Quantidade de tweets por bairro",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_range=(-4,4), y_range=(-4,4))
#disribuição normalizada
p3 = figure(plot_height=300, plot_width=600, title="Quantidade de tweets por bairro normaliada",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime', x_range=p1.x_range)
p3.xaxis.axis_label = "dias registrados"
p3.yaxis.axis_label = "quantidade de tweets"
#prueba
p4 = figure(plot_height=300, plot_width=600, title="Quantidade de tweets por bairro",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime', x_range=p1.x_range)
 

names = ["Palace Hills", "Northwest", "Old Town", "Safe Town", "Southwest", "Downtown",
         "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "Terrapin Springs",
         "Pepper Mill", "Cheddarford", "Easton", "Weston", "Southton", "Oak Willow",
         "East Parton", "West Parton"]
#Barras de interacción
bairro_init = "Palace Hills"
select_vec = Select(title="Bairro:", value=bairro_init, options=names)
s_tipo = Select(title="horas depois do terremoto:", value="5", options=["5","30"])
#histograma geral sem normalizar
edges1, hist1 = histogram(data, "location", bins=100)
source1 = ColumnDataSource(dict(hist=hist1,left=edges1[:-1], right=edges1[1:]))
p1.quad(top="hist", bottom=0, left="left", right="right", color = "blue",
            line_color="white", alpha=0.7, source=source1, legend="tweets")
p1.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]
p1.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p1.hover.mode = "vline"

#histograma geral normalizado
edges2, hist2 = histogram(data_norm.loc[data_norm.location==bairro_init], "location", bins=100)
source2 = ColumnDataSource(dict(hist=hist2,left=edges2[:-1], right=edges2[1:]))
p3.quad(top="hist", bottom=0, left="left", right="right", color = "red",
            line_color="white", alpha=0.7, source=source2, legend="tweets")
p3.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]
p3.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p3.hover.mode = "vline"

#Diagrama de pizza da cuantidade de chamadas por bairro
data_aux = data.copy()
data_aux.drop(["account","message"], axis=1)
data_aux["values"] = 1
data_aux = data_aux.groupby("location").sum()
data_aux = data_aux.drop(["UNKNOWN", "<Location with-held due to contract>"])
source3 = ColumnDataSource(dict(data_aux, angle=data_aux["values"]/(data_aux["values"]).sum() * 2*np.pi,
                                  color=Category20[19], location=data_aux.index))
p2.wedge(x=0, y=0, radius=3,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         line_color="black", fill_color='color', source=source3)
p2.hover.tooltips = [("bairro", "@location"),
                     ("quantidade", "@values")]

#histograma geral normalizado no intervalo de 5h até 30h depois do terremoto "2020-04-06 19:40:00":"2020-04-07 20:40:00"
edges2, hist2 = histogram(data_norm.loc[data_norm.location==bairro_init].loc["2020-04-06 14:40:00":"2020-04-06 19:40:00"],
                          "location", bins=100)
source2 = ColumnDataSource(dict(hist=hist2,left=edges2[:-1], right=edges2[1:]))
p4.quad(top="hist", bottom=0, left="left", right="right", color = "red",
            line_color="white", alpha=0.7, source=source2, legend="tweets")
p4.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]
p4.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p4.hover.mode = "vline"

def update_data(attrname, old, new):

    #Para obtener los valores actuales
    #day_val = s_day.value
    vec_val = select_vec.value
    #hr_val = s_range.value
    #Actualizar grafico 3
    edges2, hist2 = histogram(data_norm.loc[data_norm.location==vec_val], "location", bins=100)
    source2.data = dict(hist=hist2,left=edges2[:-1], right=edges2[1:])
    
    
#Para hacer las actualizaciones
for w in [select_vec, s_tipo]:
    w.on_change('value', update_data)

inputs = column([select_vec, s_tipo], width=200)
row_1 = row([inputs, p1, p2], width=1100)
row_2 = row([p3])
row_3 = row([])
row_4 = row([])

curdoc().add_root(row(gridplot([[row_1],[row_2],[row_3],[row_4]]), width=400))


In [16]:
cd /home/ejrueda/

/bin/sh: 1: run: not found
